In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)


import numpy as np
import math
import pickle
import folium
import json

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from xgboost import XGBClassifier


from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, FunctionTransformer, OrdinalEncoder
from sklearn.dummy import DummyRegressor
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression, BayesianRidge, Ridge
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

In [4]:
model_df = pd.read_csv("../data/model_df.csv")

In [5]:
model_df

,playerID,full_name,salary,DebutAge,yearID_year1,G_year1,AB_year1,R_year1,HR_year1,RBI_year1,SB_year1,AVG_year1,OBP_year1,SLG_year1,OPS_year1,K%_year1,BB%_year1,SB%_year1,allstar_year1,allstar_starter_year1,won_award_year1,award_votes_year1,DivWin_year1,WCWin_year1,LgWin_year1,WSWin_year1,yearID_year2,G_year2,AB_year2,R_year2,HR_year2,RBI_year2,SB_year2,AVG_year2,OBP_year2,SLG_year2,OPS_year2,K%_year2,BB%_year2,SB%_year2,allstar_year2,allstar_starter_year2,won_award_year2,award_votes_year2,DivWin_year2,WCWin_year2,LgWin_year2,WSWin_year2,yearID_year3,G_year3,AB_year3,R_year3,HR_year3,RBI_year3,SB_year3,AVG_year3,OBP_year3,SLG_year3,OPS_year3,K%_year3,BB%_year3,SB%_year3,allstar_year3,allstar_starter_year3,won_award_year3,award_votes_year3,DivWin_year3,WCWin_year3,LgWin_year3,WSWin_year3,yearID_year4,G_year4,AB_year4,R_year4,HR_year4,RBI_year4,SB_year4,AVG_year4,OBP_year4,SLG_year4,OPS_year4,K%_year4,BB%_year4,SB%_year4,allstar_year4,allstar_starter_year4,won_award_year4,award_votes_year4,DivWin_year4,WCWin_year4,LgWin_year4,WSWin_year4,yearID_year5,G_year5,AB_year5,R_year5,HR_year5,RBI_year5,SB_year5,AVG_year5,OBP_year5,SLG_year5,OPS_year5,K%_year5,BB%_year5,SB%_year5,allstar_year5,allstar_starter_year5,won_award_year5,award_votes_year5,DivWin_year5,WCWin_year5,LgWin_year5,WSWin_year5,yearID_year6,G_year6,AB_year6,R_year6,HR_year6,RBI_year6,SB_year6,AVG_year6,OBP_year6,SLG_year6,OPS_year6,K%_year6,BB%_year6,SB%_year6,allstar_year6,allstar_starter_year6,won_award_year6,award_votes_year6,DivWin_year6,WCWin_year6,LgWin_year6,WSWin_year6
0,olerujo01,John Olerud,5250000.0,21.0,1990,111,358,43,14,48.0,0.0,0.265363,0.358314,0.430168,0.788481,17.564403,14.754098,0.000000,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1991,139,454,64,17,68.0,0.0,0.255507,0.345455,0.438326,0.783781,15.272727,14.000000,0.000000,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1992,138,458,68,16,66.0,1.0,0.283843,0.366788,0.449782,0.816570,11.131387,14.781022,100.000000,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1993,158,551,109,24,107.0,0.0,0.362976,0.450843,0.598911,1.049754,9.129213,20.646067,0.000000,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1994,108,384,47,12,67.0,1.0,0.296875,0.382796,0.476562,0.859358,11.397849,15.698925,33.333333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1995,135,492,72,8,54.0,0.0,0.290650,0.390863,0.404472,0.795334,9.137056,15.905245,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,guilljo01,Jose Guillen,1575000.0,21.0,1997,143,498,58,14,70.0,1.0,0.267068,0.300380,0.411647,0.712027,16.730038,3.231939,33.333333,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1998,153,573,60,14,84.0,3.0,0.267016,0.297521,0.413613,0.711133,16.528926,3.471074,37.500000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1999,47,168,24,2,13.0,0.0,0.244048,0.310160,0.339286,0.649446,19.251337,5.882353,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2000,105,316,40,10,41.0,3.0,0.253165,0.317143,0.430380,0.747523,18.571429,5.428571,75.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2001,41,135,14,3,11.0,2.0,0.274074,0.312925,0.377778,0.690703,17.687075,5.442177,40.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2002,54,131,13,4,15.0,3.0,0.229008,0.274648,0.351145,0.625793,17.605634,5.633803,42.857143,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,perryhe01,Herbert Perry,0.0,25.0,1995,52,162,23,3,23.0,1.0,0.314815,0.369565,0.462963,0.832528,15.217391,7.065217,25.000000,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1999,66,209,29,6,32.0,0.0,0.253589,0.329167,0.397129,0.726296,17.500000,7.083333,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2000,109,383,69,12,61.0,4.0,0.308094,0.353919,0.483029,0.836948,16.152019,5.463183,80.000000,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2001,92,285,38,7,32.0,2.0,0.256140,0.324921,0.410526,0.735447,17.350158,7.570978,50.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2002,132,450,64,22,77.0,4.0,0.275556,0.329980,0.480000,0.809980,13.279678,7.042254,66.666667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2004,49,134,13,5,17.0,0.0,0.223881,0.307190,0.365672,0.672861,12.418301,9.150327,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,simmoan01,Andrelton Simmons,11500000.0,23.0,2012,49,166,17,3,19.0,1.0,0.289157,0.333333,0